# Chess Position Evaluator

Chess has been a classic benchmark for Artifical Intelligence, Machine Learning, and Data Mining since their inception. Due to the simple deterministic rules of chess coupled with the exponential possibilities and difficult to evaluate positions, it has gained the focus of many researchers in Computer Science. Today, there are several sophisticated chess AIs that compete on a level above even the strongest chess Grand Masters in the world. While chess engines like stockfish (which this dataset is based off of) and AlphaGo are too complex to try to compete with here, lets see if we can use the evaluations of Stockfish to build a comperable position evaluation function.

For this project I will be using the pytorch library and building a Neural Network with it.

In [1]:
# pytorch libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# for visualizing the results
import numpy as np
import matplotlib.pyplot as plt

# for reading input data
import pandas as pd

# for parsing the FEN of chess positions
import re


To represent a chess position, it is common to use [Forsyth–Edwards Notation (FEN)](http://https://en.wikipedia.org/wiki/Forsyth%E2%80%93Edwards_Notation) which contains all the necessary information to reconstruct a chess game from the current position. To make this information usable for a neural network, we will use a bit (actually a byte) to represent if a specific piece (white rook, white knight, etc...) is on a specific square on the 8x8 chess board. Since there are 6 different pieces and two different players, that means there are 12 specific pieces that could potentially be on each square. 

However, we still need to keep track of information like whose turn it is, which castling options are still legal, if en passant is possible, how many half moves since a pawn move or piece capture, and how many turns the game has had. To do this we use an additional 8x8 board where the rook locations represent castling rights, the 3rd and 6th rank (row) keep track of possible en passant moves, the e1 and e8 sqaure represent whose on move, and the 4th and 5th rank represent the number of half moves and full moves as binary numbers (max possible being 255) respectively.

Below is a function to do this conversion.

In [2]:
def fen_to_bit_vector(fen):
    # piece placement - lowercase for black pieces, uppercase for white pieces. numbers represent consequtive spaces. / represents a new row 
    # active color - whose turn it is, either 'w' or 'b'
    # castling rights - which castling moves are still legal K or k for kingside and Q or q for queenside, '-' if no legal castling moves for either player
    # en passant - if the last move was a pawn moving up two squares, this is the space behind the square for the purposes of en passant
    # halfmove clock - number of moves without a pawn move or piece capture, after 50 of which the game is a draw
    # fullmove number - number of full turns starting at 1, increments after black's move

    # Example FEN of starting position
    # rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
    
    parts = re.split(" ", fen)
    piece_placement = re.split("/", parts[0])
    active_color = parts[1]
    castling_rights = parts[2]
    en_passant = parts[3]
    halfmove_clock = int(parts[4])
    fullmove_clock = int(parts[5])

    bit_vector = np.zeros((13, 8, 8), dtype=np.uint8)
    
    # piece to layer structure taken from reference [1]
    piece_to_layer = {
        'R': 1,
        'N': 2,
        'B': 3,
        'Q': 4,
        'K': 5,
        'P': 6,
        'p': 7,
        'k': 8,
        'q': 9,
        'b': 10,
        'n': 11,
        'r': 12
    }
    
    castling = {
        'K': (7,7),
        'Q': (7,0),
        'k': (0,7),
        'q': (0,0),
    }

    for r, row in enumerate(piece_placement):
        c = 0
        for piece in row:
            if piece in piece_to_layer:
                bit_vector[piece_to_layer[piece], r, c] = 1
                c += 1
            else:
                c += int(piece)
    
    if en_passant != '-':
        bit_vector[0, ord(en_passant[0]) - ord('a'), int(en_passant[1]) - 1] = 1
    
    if castling_rights != '-':
        for char in castling_rights:
            bit_vector[0, castling[char][0], castling[char][1]] = 1
    
    if active_color == 'w':
        bit_vector[0, 7, 4] = 1
    else:
        bit_vector[0, 0, 4] = 1

    if halfmove_clock > 0:
        c = 7
        while halfmove_clock > 0:
            bit_vector[0, 3, c] = halfmove_clock%2
            halfmove_clock = halfmove_clock // 2
            c -= 1
            if c < 0:
                break

    if fullmove_clock > 0:
        c = 7
        while fullmove_clock > 0:
            bit_vector[0, 4, c] = fullmove_clock%2
            fullmove_clock = fullmove_clock // 2
            c -= 1
            if c < 0:
                break

    return bit_vector



In [3]:
fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"
board = fen_to_bit_vector(fen)
print(board)


[[[1 0 0 0 0 0 0 1]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 1]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [1 0 0 0 1 0 0 1]]

 [[0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 1]]

 [[0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 1 0 0 0 0 1 0]]

 [[0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 1 0 0 1 0 0]]

 [[0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 1 0 0 0 0]]

 [[0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 1 0 0 0]]

 [[0 0 0 0 0 0 0 0]
  [0 0 0

The first 8x8 board (0th index) contains all the extra information and the following 12 boards (1 to 12) represent the locations of the pieces in the order 

1. White Rook
2. White Knight
3. White Bishop
4. White Queen
5. White King
6. White Pawn
7. Black Pawn
8. Black King
9. Black Queen
10. Black Bishop
11. Black Knight
12. Black Rook

Notice how the pieces line up correctly with the starting position with the first board correctly indicating it is white to move.

# Neural Network

We'll begin with a simple Feed-Forward Neural Network that's fully connected. Neural Networks are named for their structure being analogous to neurons in the human brain. The idea is that, in the human brain, when a neuron gets an electrical impulse through its synapses it will sometimes fire an electrical impulse to other neurons, creating a chain reaction. For neural networks, our neurons are nodes our synapses are edges (with corresponding weights) and the firing of the neuron is the activation function and output of the node. 

![Perceptron](http://starship-knowledge.com/wp-content/uploads/2020/10/Perceptrons-1024x724.jpeg)

The goal of the Neural Network is to have weights such that after all the chain reactions of nodes taking in inputs and producing outputs, the information output of the final node represents the evaluation of the chess position that began the process. In order to actually find such weights we will use a method known as backpropagation, which iteratively adjusts the weights in the network to nudge the output closer to the answer we desire.

Technically speaking, for each training record (a FEN and an evaulation) we input the position into the Neural Network, and after we get a result we compute the error between the result and the correct evaluation which can be represented as a error function. To change the weights in such a way as to minimize this error function we compute the gradient of the error function and adjust the weights in the opposite direction. This means that if we overshoot we want to decrease our evaluation and if we undershoot we want to increase our evaluation.

![Gradient Descent](https://sebastianraschka.com/images/blog/2015/singlelayer_neural_networks_files/perceptron_gradient_descent_1.png)


In [4]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(832, 832)
        self.fc2 = nn.Linear(832, 416)
        self.fc3 = nn.Linear(416, 208)
        self.fc4 = nn.Linear(208, 104)
        self.fc5 = nn.Linear(104, 1)
        self.dropout = nn.Dropout(0.2)
        self.lrelu = nn.LeakyReLU()


    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.lrelu(x)
        
        x = self.fc2(x)
        x = self.dropout(x)
        x = self.lrelu(x)
        
        x = self.fc3(x)
        x = self.dropout(x)
        x = self.lrelu(x)
        
        x = self.fc4(x)
        x = self.dropout(x)
        x = self.lrelu(x)

        x = self.fc5(x)
        return x



In [5]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.lrelu = nn.LeakyReLU(0.01)


    def forward(self, x):
        x = torch.flatten(x, 1)
        
        return x


In [6]:
# ChessDataset code and eval_to_int code taken from reference [1]
class ChessDataset(Dataset):
    def __init__(self, data_frame):
        self.fens = torch.from_numpy(np.array([*map(fen_to_bit_vector, data_frame["FEN"])], dtype=np.float32))
        self.evals = torch.Tensor([[x] for x in data_frame["Evaluation"]])
        self._len = len(self.evals)
        
    def __len__(self):
        return self._len
    
    def __getitem__(self, index):
        return self.fens[index], self.evals[index]


def eval_to_int(evaluation):
    try:
        res = int(evaluation)
    except ValueError:
        res = 10000 if evaluation[1] == '+' else -10000
    return res / 100



In [11]:
epochs = 20
batch_size = 100
MAX_DATA = 500000

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device {}".format(device))

def AdamW_main():
    print("Preparing Training Data...")
    train_data = pd.read_csv("chessData.csv")
    train_data = train_data[:MAX_DATA]
    train_data["Evaluation"] = train_data["Evaluation"].map(eval_to_int)
    trainset = ChessDataset(train_data)
    
    print("Preparing Test Data...")
    test_data = pd.read_csv("tactic_evals.csv")
    test_data = test_data[:MAX_DATA]
    test_data["Evaluation"] = test_data["Evaluation"].map(eval_to_int)
    testset = ChessDataset(test_data)

    print("Converting to pytorch Dataset...")

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)


    net = Net().to(device)
    criterion = nn.MSELoss()
    optimizer = optim.AdamW(net.parameters())


    for epoch in range(epochs):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 500 == 499:    # print every 2000 mini-batches
                # denominator for loss should represent the number of positions evaluated 
                # independent of the batch size
                print('[%d, %d] loss: %.3f' % (epoch + 1, i + 1, running_loss / (500*len(labels))))
                running_loss = 0.0

    print('Finished Training')

    PATH = './chess_fc_dropout_1mil.pth'
    torch.save(net.state_dict(), PATH)

    print('Evaluating model')

    count = 0
    total_loss = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            
            # count should represent the number of positions evaluated 
            # independent of the batch size
            count += len(labels)
            total_loss += loss
            if count % 10000 == 0:
                print('Average error of the model on the {} tactics positions is {}'.format(count, loss/count))
    #print('Average error of the model on the {} tactics positions is {}'.format(count, loss/count))


Using device cuda


In [12]:
AdamW_main()

Preparing Training Data...
Preparing Test Data...
Converting to pytorch Dataset...


KeyboardInterrupt: 

# Results

An average error of 0.0225 seems very reasonable, however this is not as amazing as it might at first seem since positions where stockfish's evaluation is not forced mate have been normalized to a range from approximately -1.5 to 1.5 and positions where stockfish's evalutaion is forced mate have been set to -100 or 100. By convention a positive score indicates an advantage for white and a negative score indicates an advantage for black.

# Testing Optimization Algorithms

Originally we used the AdamW algorithm for the optimization step, but what if we try Stochastic Gradient Descent (SGD)?

In [ ]:
def SGD_main():
    MAX_DATA = 100000
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device {}".format(device))

    print("Preparing Training Data...")
    train_data = pd.read_csv("/kaggle/input/chess-evaluations/chessData.csv")
    train_data = train_data[:MAX_DATA]
    train_data["Evaluation"] = train_data["Evaluation"].map(eval_to_int)
    trainset = ChessDataset(train_data)
    
    print("Preparing Test Data...")
    test_data = pd.read_csv("/kaggle/input/chess-evaluations/tactic_evals.csv")
    test_data = test_data[:MAX_DATA]
    test_data["Evaluation"] = test_data["Evaluation"].map(eval_to_int)
    testset = ChessDataset(test_data)

    batch_size = 10

    print("Converting to pytorch Dataset...")

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)


    net = Net().to(device)
    criterion = nn.MSELoss()
    optimizer = optim.SGD(net.parameters(), lr = 0.01)


    for epoch in range(10):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                # denominator for loss should represent the number of positions evaluated 
                # independent of the batch size
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / (2000*len(labels))))
                running_loss = 0.0

    print('Finished Training')

    PATH = './chess.pth'
    torch.save(net.state_dict(), PATH)

    print('Evaluating model')

    count = 0
    total_loss = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            #print("Correct eval: {}, Predicted eval: {}, loss: {}".format(labels, outputs, loss))
            
            # count should represent the number of positions evaluated 
            # independent of the batch size
            count += len(labels)
            total_loss += loss
            if count % 10000 == 0:
                print('Average error of the model on the {} tactics positions is {}'.format(count, loss/count))
    #print('Average error of the model on the {} tactics positions is {}'.format(count, loss/count))


In [ ]:
SGD_main()

Using device cuda
Preparing Training Data...


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/chess-evaluations/chessData.csv'

An average error of 0.0207 is a slight improvement, but not so much that we can definitively say that SGD did better than AdamW. Since the training loss did not decrease by much throughout the 10 epochs, it's much more likely that either SGD is a worse optimizer for this classification problem or the learning rate is too large which is preventing the model from converging to the local optimum. Let's try again but with a learning rate of 0.001

In [ ]:
def SGD_lr_main(learning_rate):
    MAX_DATA = 100000
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device {}".format(device))

    print("Preparing Training Data...")
    train_data = pd.read_csv("/kaggle/input/chess-evaluations/chessData.csv")
    train_data = train_data[:MAX_DATA]
    train_data["Evaluation"] = train_data["Evaluation"].map(eval_to_int)
    trainset = ChessDataset(train_data)
    
    print("Preparing Test Data...")
    test_data = pd.read_csv("/kaggle/input/chess-evaluations/tactic_evals.csv")
    test_data = test_data[:MAX_DATA]
    test_data["Evaluation"] = test_data["Evaluation"].map(eval_to_int)
    testset = ChessDataset(test_data)

    batch_size = 10

    print("Converting to pytorch Dataset...")

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)


    net = Net().to(device)
    criterion = nn.MSELoss()
    optimizer = optim.SGD(net.parameters(), lr = learning_rate)


    for epoch in range(10):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                # denominator for loss should represent the number of positions evaluated 
                # independent of the batch size
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / (2000*len(labels))))
                running_loss = 0.0

    print('Finished Training')

    PATH = './chess.pth'
    torch.save(net.state_dict(), PATH)

    print('Evaluating model')

    count = 0
    total_loss = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            #print("Correct eval: {}, Predicted eval: {}, loss: {}".format(labels, outputs, loss))
            
            # count should represent the number of positions evaluated 
            # independent of the batch size
            count += len(labels)
            total_loss += loss
            if count % 10000 == 0:
                print('Average error of the model on the {} tactics positions is {}'.format(count, loss/count))
    #print('Average error of the model on the {} tactics positions is {}'.format(count, loss/count))


In [ ]:
SGD_lr_main(0.001)

Using device cuda
Preparing Training Data...
Preparing Test Data...
Converting to pytorch Dataset...
[1,  2000] loss: 8.440
[1,  4000] loss: 9.587
[1,  6000] loss: 9.200
[1,  8000] loss: 9.099
[1, 10000] loss: 10.472
[2,  2000] loss: 9.207
[2,  4000] loss: 9.295
[2,  6000] loss: 8.579
[2,  8000] loss: 8.531
[2, 10000] loss: 8.224
[3,  2000] loss: 8.834
[3,  4000] loss: 6.915
[3,  6000] loss: 7.801
[3,  8000] loss: 9.602
[3, 10000] loss: 8.839
[4,  2000] loss: 8.003
[4,  4000] loss: 7.340
[4,  6000] loss: 7.545
[4,  8000] loss: 7.403
[4, 10000] loss: 8.222
[5,  2000] loss: 7.645
[5,  4000] loss: 8.381
[5,  6000] loss: 7.654
[5,  8000] loss: 6.861
[5, 10000] loss: 8.815
[6,  2000] loss: 7.837
[6,  4000] loss: 6.889
[6,  6000] loss: 7.066
[6,  8000] loss: 7.349
[6, 10000] loss: 7.577
[7,  2000] loss: 6.622
[7,  4000] loss: 7.293
[7,  6000] loss: 6.500
[7,  8000] loss: 6.772
[7, 10000] loss: 7.374
[8,  2000] loss: 6.713
[8,  4000] loss: 6.339
[8,  6000] loss: 6.384
[8,  8000] loss: 5.813
[

The training error seems like it's decreasing even if sporadic, but the classification error got slightly worse. It seems like AdamW is simply more consistant and converges faster to a local optimum.

# Conclusion

AdamW seems to be a more consistant algorithm and one which converges more quickly to a local optimum based on the comparision of AdamW, SGD(lr=0.01), and SGD(lr=0.001)

This means the best model had an average error of 0.0225 as opposed to the lowest average error of 0.0207 which is not significant enough of a difference to justify the inconsistancy of the SGD optimizer.

# Challenges

This dataset (or at least the chessData.csv dataset) has over 16 million positions and evaluations which makes it rather difficult to work with. Because of this I've had to limit the amount of data actually used to 200,000 positions, 100,000 in the training dataset and 100,000 in the test dataset. 

# References

1. https://www.kaggle.com/ronakbadhe/chess-evaluation-prediction
2. https://en.wikipedia.org/wiki/Forsyth%E2%80%93Edwards_Notation
3. http://starship-knowledge.com/wp-content/uploads/2020/10/Perceptrons-1024x724.jpeg
4. https://starship-knowledge.com/wp-content/uploads/2020/10/Perceptrons-1024x724.jpeg

# Contribution

* Representation of castling rights, en passant, active color, halfmoves and fullmoves in an 8x8 grid
* Neural Network Architecture
* Testing AdamW vs SVG